In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

cannot find .env file


In [3]:
import sys
import os
from pathlib import Path

sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(Path(os.getcwd()).parent.absolute())

import mcore as mc
#from common import *
print("dir", dir(mc))
print("all", mc.__all__)


dir ['CoreObject', 'Neuron', 'RunnableObject', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'core_object', 'neuron']
all ['CoreObject', 'Neuron', 'RunnableObject']


In [5]:
([0.]*18) +  [2.] + ([0.]*18) +  [2.]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0]

In [4]:
# Neuron Test
n = mc.Neuron()
n.energyLeak = .1
n.peakEnergy = 2.
for i in range(50):
    n.onSignal(i, .2)
    tick_rc = n.onTick(i)
    print(f"iter({i}) {n} ticks with {tick_rc} result ")

iter(0) N[0]e:0.1 l:0.1 p:2.0 ticks with 0.0 result 
iter(1) N[0]e:0.20000000000000004 l:0.1 p:2.0 ticks with 0.0 result 
iter(2) N[0]e:0.30000000000000004 l:0.1 p:2.0 ticks with 0.0 result 
iter(3) N[0]e:0.4 l:0.1 p:2.0 ticks with 0.0 result 
iter(4) N[0]e:0.5000000000000001 l:0.1 p:2.0 ticks with 0.0 result 
iter(5) N[0]e:0.6000000000000002 l:0.1 p:2.0 ticks with 0.0 result 
iter(6) N[0]e:0.7000000000000003 l:0.1 p:2.0 ticks with 0.0 result 
iter(7) N[0]e:0.8000000000000004 l:0.1 p:2.0 ticks with 0.0 result 
iter(8) N[0]e:0.9000000000000005 l:0.1 p:2.0 ticks with 0.0 result 
iter(9) N[0]e:1.0000000000000004 l:0.1 p:2.0 ticks with 0.0 result 
iter(10) N[0]e:1.1000000000000003 l:0.1 p:2.0 ticks with 0.0 result 
iter(11) N[0]e:1.2000000000000002 l:0.1 p:2.0 ticks with 0.0 result 
iter(12) N[0]e:1.3 l:0.1 p:2.0 ticks with 0.0 result 
iter(13) N[0]e:1.4 l:0.1 p:2.0 ticks with 0.0 result 
iter(14) N[0]e:1.4999999999999998 l:0.1 p:2.0 ticks with 0.0 result 
iter(15) N[0]e:1.5999999999999996